In [1]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd
import numba
from numba import njit
import datetime
import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.optimize as spop
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime as daee
from itertools import combinations
import json
import requests


In [14]:
end = datetime.datetime.now()
start = end - datetime.timedelta(days=365*10)
vbt.settings.set_theme("dark")

In [3]:
data = vbt.YFData.pull(
    ["F","GM"],
    start=start,
    end=end,
)   
display(data.shape)

  0%|          | 0/2 [00:00<?, ?it/s]

(2515, 2)

In [4]:
open_price=data.get("Open")["F"]
close_price=data.get("Close")["F"]

In [5]:
def ma_strategy(close_price,window=5,lower_multiple=1,upper_multiple=4):
    signal=np.full(close_price.shape[0],np.nan)
    for x in range(len(close_price)):
        if x >=window:
            ma=np.mean(close_price[x-window:x])
            if close_price[x]<ma*lower_multiple:
                signal[x]=1
            elif close_price[x]>ma*upper_multiple:
                signal[x]=-1
            else:
                signal[x]=0
    return signal

In [6]:
indicator= vbt.IndicatorFactory(
    class_name="Clubbed",
    short_name="Clu",
    input_names=["close_price"],
    param_names=["window","lower_multiple","upper_multiple"],
    output_names=["signal"],
).with_apply_func(
    ma_strategy,
    window=10,
    lower_multiple=1,
    upper_multiple=4,)

In [7]:
results = indicator.run(close_price)
results.signal

Date
2013-11-07 00:00:00-05:00    NaN
2013-11-08 00:00:00-05:00    NaN
2013-11-11 00:00:00-05:00    NaN
2013-11-12 00:00:00-05:00    NaN
2013-11-13 00:00:00-05:00    NaN
                            ... 
2023-10-30 00:00:00-04:00    1.0
2023-10-31 00:00:00-04:00    1.0
2023-11-01 00:00:00-04:00    1.0
2023-11-02 00:00:00-04:00    1.0
2023-11-03 00:00:00-04:00    1.0
Name: F, Length: 2515, dtype: float64

In [8]:
entries= results.signal ==1
exits =results.signal  ==-1

In [9]:
portfolio=vbt.Portfolio.from_signals(close_price,entries,exits,fees=0.001,init_cash=100000) 


In [10]:
portfolio.stats()

c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\generic\stats_builder.py:463: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\generic\stats_builder.py:463: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\generic\stats_builder.py:463: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\generic\stats_builder.py:463: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\base\wrapping.py:971: UserWarning: Couldn't parse the frequency of index. Pass it as `freq` or define it globally under `settings.wrapping`.
  return self.index_acc.ar

Start                         2013-11-07 00:00:00-05:00
End                           2023-11-03 00:00:00-04:00
Period                                             2515
Start Value                                    100000.0
Min Value                                  33359.845002
Max Value                                 210614.927095
End Value                                 100744.260102
Total Return [%]                                0.74426
Benchmark Return [%]                           3.221437
Total Time Exposure [%]                       99.522863
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              68.787488
Max Drawdown Duration                            1668.0
Total Orders                                          1
Total Fees Paid                                 99.9001
Total Trades                                          1
Win Rate [%]                                        NaN
Best Trade [%]                                  

In [15]:
portfolio.value.vbt.plot()

FigureWidget({
    'data': [{'name': 'F',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'af28d31f-0aa8-4071-a67a-c0d8be512bbd',
              'x': array([datetime.datetime(2013, 11, 7, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2013, 11, 8, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2013, 11, 11, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2023, 11, 1, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2023, 11, 2, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2023, 11, 3, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)],
                         dtype=obje